In [1]:
import re
from unidecode import unidecode
import pandas as pd
from ibge.localidades import Municipios

cidades = Municipios().json()
cidades = {cidade['nome']: cidade['microrregiao']['mesorregiao']['UF']['sigla'] for cidade in cidades}

df = pd.read_excel('dados/linkedin.xlsx')
df.head()

,Data,Empresa,Título,Localização,Descrição,Nível,Tipo,Função,Indústria,Link
0,2023-01-06,Vigor,Analista de Dados Jr,"São Paulo, SP",Oportunidade de compor o time de Tecnologia da...,Não aplicável,Tempo integral,Tecnologia da informação,Seguros,https://br.linkedin.com/jobs/view/analista-de-...
1,2023-01-06,Vigor,Analista de Dados Jr,"São Paulo, SP",Inovação e Desafios são o nosso foco!A nossa C...,Pleno-sênior,Tempo integral,Tecnologia da informação,Serviços de alimentos e bebidas,https://br.linkedin.com/jobs/view/analista-de-...
2,2022-11-22,BIX Tecnologia - Consultoria de Dados,Cientista de Dados - Junior (Remoto),"Florianópolis, SC",Inovação e Desafios são o nosso foco!A nossa C...,Júnior,Tempo integral,Estratégia/Planejamento,Serviços de alimentos e bebidas,https://br.linkedin.com/jobs/view/cientista-de...
3,2023-01-07,americanas s.a.,Analista de Dados Jr,"Nova Iguaçu, RJ","Somos uma empresa apaixonada por pessoas, paut...",Não aplicável,Tempo integral,Engenharia e Tecnologia da informação,Serviços e consultoria de TI,https://br.linkedin.com/jobs/view/analista-de-...
4,2022-12-11,MadeiraMadeira,Engenheira de Dados JR,"Curitiba, PR",Job DescriptionNossa missão é fazer mais e mel...,Pleno-sênior,Tempo integral,Tecnologia da informação,Varejo,https://br.linkedin.com/jobs/view/engenheira-d...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Data         834 non-null    object
 1   Empresa      834 non-null    object
 2   Título       834 non-null    object
 3   Localização  834 non-null    object
 4   Descrição    834 non-null    object
 5   Nível        834 non-null    object
 6   Tipo         834 non-null    object
 7   Função       834 non-null    object
 8   Indústria    834 non-null    object
 9   Link         834 non-null    object
dtypes: object(10)
memory usage: 65.3+ KB


In [3]:
df['Data'] = pd.to_datetime(df['Data'], errors='coerce', format='%Y-%m-%d')

In [4]:
df['Título'].unique()

array(['Analista de Dados Jr', 'Cientista de Dados - Junior (Remoto)',
       'Engenheira de Dados JR', 'Analista de Riscos Júnior',
       'Analista de Dados Júnior', 'Analista de Ciência de Dados Junior',
       'ANALISTA DE NEGOCIOS JR', 'Analista de Dados',
       'Pessoa Estagiária em Data Science',
       'Pessoa Estagiária em Data Science & IA', 'Estágio de Dados',
       'Estágio - Data Science RJ', 'Analista de Dados JR',
       'Cientista de Dados Júnior', 'Estagiário(a) Ciência de Dados',
       'BUSINESS ANALYTICS JUNIOR ( 100% REMOTO)',
       'ANALISTA DE DADOS (Remoto)', 'Analista de dados',
       'Analista de Dados/ Crédito',
       'Analista de Dados Administrativo Jr', 'Estágio - Data Science NE',
       'Cientista de dados', 'Estágio - Data Science SP',
       'Analista de Planejamento Jr', 'Analista de Dados jr',
       'Analista Segurança da Informação SOC - Jr',
       'Analista de sistemas Jr', 'ANALISTA DADOS I',
       'Engenheiro de dados Jr', 'Analista de Ci

In [5]:
def identifica_cargo(titulo):
    kwmap = {
        'Cientista de Dados': [('cientista', 'dados'), ('data', 'scientist'), ('data', 'science'), ('ciencia', 'dados')],
        'Analista de Dados': [('analista', 'dados'), ('data', 'analyst'), ('analise', 'dados')],
        'Analista de Negócios': [('analista', 'negocio'), ('business', 'analyst'), ('business', 'analytics')],
        'Engenheiro(a) de Dados': [('engenheiro', 'dados'), ('data', 'engineer'), ('engenharia', 'dados'), ('engenheira', 'dados')],
        'Assistente de Dados': [('assistente', 'dados')],
        'Auxiliar de Dados': [('auxiliar', 'dados')]
    }
    
    words = unidecode(titulo).lower().split(' ')
    
    for cargo, kws in kwmap.items():
        for kw in kws:
            exists = [True if w in words else False for w in kw]
            if all(exists):
                return cargo
            
    return pd.NA

df['Cargo'] = df['Título'].apply(identifica_cargo)
df['Cargo'].value_counts(sort=True, ascending=False)

Analista de Dados         230
Cientista de Dados         63
Engenheiro(a) de Dados     31
Assistente de Dados         6
Analista de Negócios        3
Auxiliar de Dados           2
Name: Cargo, dtype: int64

In [6]:
df.drop(df[df['Cargo'].isnull()].index, axis=0, inplace=True)
df.shape

(335, 11)

In [7]:
df['Localização'].unique()

array(['São Paulo, SP', 'Florianópolis, SC', 'Nova Iguaçu, RJ',
       'Curitiba, PR', 'Rio de Janeiro, RJ', 'Brasil',
       'Porto Alegre e Região', 'Belo Horizonte, MG', 'Agudos, SP',
       'Vinhedo, SP', 'Paracatu, MG', 'Brasília, DF', 'Barueri, SP',
       'Recife e Região', 'Recife, PE', 'Jandira, SP', 'Valinhos, SP',
       'São Paulo, Brasil', 'Sorocaba, SP', 'Nova Lima, MG',
       'Rio Verde, GO', 'São Paulo e Região', 'São José dos Campos, SP',
       'Santa Catarina, Brasil', 'Campinas, SP', 'Porto Alegre, RS',
       'Contagem, MG', 'Piracicaba, SP', 'Cachoeirinha, RS',
       'Goiânia, GO', 'Marechal Deodoro, AL', 'Vitória, ES',
       'Rio do Sul, SC', 'Campo Grande, MS', 'Ponta Grossa, PR',
       'Rio Claro, SP', 'Juiz de Fora, MG', 'Monte Mor, SP',
       'Divinópolis, MG', 'Florianópolis e Região', 'Macaé, RJ',
       'Osasco, SP', 'Cascavel, PR', 'Campinas e Região', 'Palhoça, SC',
       'Blumenau, SC', 'São José, SC', 'Aracruz, ES', 'Fortaleza, CE',
       'Mogi 

In [8]:
def processa_localizacao(localizacao):
    regex = '^(.+)(, | e )(.+)$'
    s = re.search(regex, localizacao)
    if s:
        cidade = s.group(1)
        if cidade in cidades:
            return f'{cidade}/{cidades[cidade]}'

    return localizacao.replace(', ','/')
    
df['Localização'] = df['Localização'].apply(processa_localizacao)

In [9]:
df = df.reset_index().drop('index', axis=1)
with pd.ExcelWriter('dados/vagas.xlsx', engine='auto') as writer:
    df.to_excel(writer, 'LinkedIn', index=False)